## Instalando Bibliotecas

In [ ]:
pip install openmeteo-requests requests-cache retry-requests

## Imports
Realizando os imports necessários para usar as API's

In [1]:
import json
import requests
import pandas as pd
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

## Questão 1:
Quantos feriados há no Brasil em todo o ano de 2024?

Usei a API Nager.DATE ( Public Holidays )<br>
Aqui usei a API e laço for para contabilizar a quantidade de feriados no ano de 2024

In [2]:
response = requests.get('https://date.nager.at/api/v3/publicholidays/2024/BR')
public_holidays = json.loads(response.content)
feriados_2024 = 0

for public_holiday in public_holidays:
  feriados_2024 += 1
  
print(feriados_2024)

14


Resposta: Em 2024 teremos 14 feriados

## Questão 2:
Qual mês de 2024 tem o maior número de feriados?

Nessa primeira parte eu decidi criar um DataFrame com os feriados, para que eu pudesse ir utilizando<br>
essa base de dados durante as questões.

In [3]:
df_feriados = pd.DataFrame(columns=['Holiday Name', 'Date'])
rows = []

for public_holiday in public_holidays:
  rows.append({'Holiday Name': public_holiday['name'], 'Date': public_holiday['date'],})
  
df_feriados = pd.concat([df_feriados, pd.DataFrame(rows)], ignore_index=True)
df_feriados['Date'] = pd.to_datetime(df_feriados['Date'])
df_feriados['Month'] = df_feriados['Date'].dt.month_name()
df_feriados['Day'] = df_feriados['Date'].dt.weekday
df_feriados['Weekday'] = df_feriados['Date'].dt.day_name()

Visualização do DataFrame

In [4]:
df_feriados.head(30)

,Holiday Name,Date,Month,Day,Weekday
0,New Year's Day,2024-01-01,January,0,Monday
1,Carnival,2024-02-12,February,0,Monday
2,Carnival,2024-02-13,February,1,Tuesday
3,Good Friday,2024-03-29,March,4,Friday
4,Easter Sunday,2024-03-31,March,6,Sunday
5,Tiradentes,2024-04-21,April,6,Sunday
6,Labour Day,2024-05-01,May,2,Wednesday
7,Corpus Christi,2024-05-30,May,3,Thursday
8,Constitutionalist Revolution of 1932,2024-07-09,July,1,Tuesday
9,Independence Day,2024-09-07,September,5,Saturday


Aqui onde de fato eu fiz a captura dos meses que tinham mais feriados no ano de 2024.<br>
Usei dt.month para capturar apenas o mês da coluna Date, que foi passada para datetime anteriormente<br>
e como havia meses com a mesma quantidade de feriados, usei .max para pegar o valor máximo e printar<br>
todos que estavam no valor máximo

In [5]:
feriados_por_mes = df_feriados['Date'].dt.month.value_counts().sort_index()
max_feriados = feriados_por_mes.max()
meses_com_max_feriados = feriados_por_mes[feriados_por_mes == max_feriados]

print("Meses com o maior número de feriados:\n")
print(meses_com_max_feriados)

Meses com o maior número de feriados:

Date
2     2
3     2
5     2
11    2
Name: count, dtype: int64


Resposta: <br>
2 - Fevereiro ----- 2 feriados<br>
3 - Março ----- 2 feriados <br>
5 - Maio ----- 2 feriados<br>
11 - Novembro ----- 2 feriados<br>

## Questão 3: 
Quantos feriados em 2024 caem em dias de semana (segunda a sexta-feira)?


Aqui eu aproveitei o DataFrame e fiz um laço for para percorrer a coluna dias, e verificar<br>
quantos dias ficam entre segunda e sexta (entre 0 e 4).

In [6]:
count_dias_semana = 0

for dia in df_feriados['Day']:
    if 0 <= dia <= 4:
        count_dias_semana += 1

In [7]:
print(count_dias_semana)

9


Resposta: 9 feriados

## Questão 4:
Qual foi a temperatura média em cada mês?

Através do site: [Open Meteo](https://open-meteo.com/en/docs#latitude=-22.9064&longitude=-43.1822&hourly=&daily=temperature_2m_max,temperature_2m_min&timezone=America%2FSao_Paulo&start_date=2024-01-01&end_date=2024-08-01&time_mode=time_interval) <br>
Eu consegui achar como realizar as configurar para trabalhar com a API. Eu não encontrei no site configurações para extrair <br>
de forma direta a temperatura média do dia, dessa forma eu fiz uma aritmética simples, somando a temperatura max e min do dia<br>
e dividindo por 2, para ter uma média aproxímada da temperatura.

In [14]:
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -22.9064,
	"longitude": -43.1822,
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min"],
	"timezone": "America/Sao_Paulo",
	"start_date": "2024-01-01",
	"end_date": "2024-08-01"
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]

daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()

daily_data = {"Date": pd.date_range(
    start= pd.to_datetime(daily.Time(), unit = "s", utc = False),
    end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = False),
    freq = pd.Timedelta(seconds = daily.Interval()),
    inclusive= "left"
)}

daily_dataframe = pd.DataFrame(data = daily_data)

daily_data["weather_code"] = daily_weather_code
daily_data['Ano'] = daily_dataframe['Date'].dt.year
daily_data['Mês'] = daily_dataframe['Date'].dt.month
daily_data["Temperatura Máxima"] = daily_temperature_2m_max
daily_data["Temperatura Mínima"] = daily_temperature_2m_min
daily_data["Temperatura Média"] = (daily_data['Temperatura Máxima'] + daily_data['Temperatura Mínima']) / 2

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe["Date"] = daily_dataframe["Date"].dt.date
daily_dataframe["Date"] = pd.to_datetime(daily_dataframe['Date'])

daily_dataframe.head(195)

,Date,weather_code,Ano,Mês,Temperatura Máxima,Temperatura Mínima,Temperatura Média
0,2024-01-01,NaN,2024,1,NaN,NaN,NaN
1,2024-01-02,NaN,2024,1,NaN,NaN,NaN
2,2024-01-03,NaN,2024,1,NaN,NaN,NaN
3,2024-01-04,NaN,2024,1,NaN,NaN,NaN
4,2024-01-05,NaN,2024,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...
190,2024-07-09,3.0,2024,7,23.525999,20.226000,21.875999
191,2024-07-10,45.0,2024,7,27.575998,19.226000,23.400999
192,2024-07-11,45.0,2024,7,26.625999,21.226000,23.925999
193,2024-07-12,3.0,2024,7,24.226000,21.425999,22.826000


Continuando a resolução, aqui eu criei um DataFrame para as médias mensais

Peguei as informações do outro DataFrame e agrupei os dados usando o mês como métrica, calculando a média de temperatura para cada mês

In [9]:
df_medias_mensais = pd.DataFrame(columns=['Mês', 'Ano', 'Temperatura Média'])

df_medias_mensais = daily_dataframe.groupby(['Ano', 'Mês'])['Temperatura Média'].mean().reset_index()

df_medias_mensais.head(10)

,Ano,Mês,Temperatura Média
0,2024,1,NaN
1,2024,2,NaN
2,2024,3,NaN
3,2024,4,NaN
4,2024,5,24.158142
5,2024,6,23.961832
6,2024,7,22.242128
7,2024,8,22.000999


Resposta: <br>
Só temos informações metereológicas a partir do dia 18/05/2023, por isso não temos médias dos meses Janeiro, Fevereiro, Março e Abril.<br>
Maio: 24.15 ºC<br>
Junho: 24 ºC<br>
Julho: 22.24 ºC<br>
Agosto: 22.00 ºC<br>

## Questão 5:
Qual foi o tempo predominante em cada mês nesse período?

Aqui eu fiz uma função utilizando o método mode() para retornar o weather_code mais recorrente<br>
e agrupei usando Ano e Mês como filtro, aplicando a função em função do weather_code<br>
e fiz o merge do dataframe medias mensais e o novo dataframe criado

In [10]:
def weather_code_recorrente(valores):
    frequencia = valores.mode()
    return frequencia.iloc[0] if not frequencia.empty else None

weather_code_comum = daily_dataframe.groupby(['Ano', 'Mês'])['weather_code'].apply(weather_code_recorrente).reset_index()
weather_code_comum.rename(columns={'weather_code': 'Weather_Code_Mais_Recorrente'}, inplace=True)


df_resultado = pd.merge(df_medias_mensais, weather_code_comum, on=['Ano', 'Mês'])

df_resultado.head(10)

,Ano,Mês,Temperatura Média,Weather_Code_Mais_Recorrente
0,2024,1,NaN,NaN
1,2024,2,NaN,NaN
2,2024,3,NaN,NaN
3,2024,4,NaN,NaN
4,2024,5,24.158142,80.0
5,2024,6,23.961832,45.0
6,2024,7,22.242128,45.0
7,2024,8,22.000999,45.0


Aqui eu "carreguei" o JSON ao nosso código e fiz uma função, para fazer o mapping<br>
do weather_code do meu DataFrame com os valores que estão no JSON.

In [11]:
with open('descriptions.json', 'r') as file:
    weather_code_dict = json.load(file)

def pegar_Descrição_JSON(code, time_of_day='day'):
    if pd.isna(code):  # Verifica se o código é NaN
        return 'Descrição não disponível'
    code_str = str(int(code))
    if code_str in weather_code_dict:
        return weather_code_dict[code_str][time_of_day]['description']
    else:
        return 'Descrição não disponível'


df_resultado['Descrição do Tempo'] = df_resultado['Weather_Code_Mais_Recorrente'].apply(lambda x: pegar_Descrição_JSON(x, 'day'))

df_resultado.head(10)

,Ano,Mês,Temperatura Média,Weather_Code_Mais_Recorrente,Descrição do Tempo
0,2024,1,NaN,NaN,Descrição não disponível
1,2024,2,NaN,NaN,Descrição não disponível
2,2024,3,NaN,NaN,Descrição não disponível
3,2024,4,NaN,NaN,Descrição não disponível
4,2024,5,24.158142,80.0,Light Showers
5,2024,6,23.961832,45.0,Foggy
6,2024,7,22.242128,45.0,Foggy
7,2024,8,22.000999,45.0,Foggy


Resposta: <br>
Pelos mesmos fatores da questão 4, o 4 primeiro meses não temos informações metereólogicas<br>
por isso, temos apenas o weather_code dos últimos 4 meses.<br>
<br>
Maio - Light Showers<br>
Junho - Foggy<br>
Julho - Foggy<br>
Agosto - Foggy<br>

## Questão 6:
Qual foi o tempo e a temperatura média em cada feriado de 01/01/2024 a 01/08/2024?

In [12]:
data_inicio = '2024-01-01'
data_fim = '2024-08-01'

df_feriados_filtro = df_feriados[(df_feriados['Date'] >= data_inicio) & (df_feriados['Date'] <= data_fim)]
df_feriados_filtro.head(10)


,Holiday Name,Date,Month,Day,Weekday
0,New Year's Day,2024-01-01,January,0,Monday
1,Carnival,2024-02-12,February,0,Monday
2,Carnival,2024-02-13,February,1,Tuesday
3,Good Friday,2024-03-29,March,4,Friday
4,Easter Sunday,2024-03-31,March,6,Sunday
5,Tiradentes,2024-04-21,April,6,Sunday
6,Labour Day,2024-05-01,May,2,Wednesday
7,Corpus Christi,2024-05-30,May,3,Thursday
8,Constitutionalist Revolution of 1932,2024-07-09,July,1,Tuesday


In [15]:
df_resultado_feriados = pd.merge(df_feriados_filtro, daily_dataframe[['Date', 'Temperatura Média']], on='Date', how='left')

In [16]:
df_resultado_feriados.tail(10)

,Holiday Name,Date,Month,Day,Weekday,Temperatura Média
0,New Year's Day,2024-01-01,January,0,Monday,NaN
1,Carnival,2024-02-12,February,0,Monday,NaN
2,Carnival,2024-02-13,February,1,Tuesday,NaN
3,Good Friday,2024-03-29,March,4,Friday,NaN
4,Easter Sunday,2024-03-31,March,6,Sunday,NaN
5,Tiradentes,2024-04-21,April,6,Sunday,NaN
6,Labour Day,2024-05-01,May,2,Wednesday,NaN
7,Corpus Christi,2024-05-30,May,3,Thursday,21.550999
8,Constitutionalist Revolution of 1932,2024-07-09,July,1,Tuesday,21.875999


In [ ]:
print(df_feriados_filtro['Date'].dtype)